# Quebrando captchas vulneráveis - Visão computacional

---

Nesse notebook veremos os princípios utilizados e os passos necessários para *quebrar CAPTCHAs*, desde o processamento de imagens até extração de informações . Para isso, utilizaremos as bilbiotecas: Selenium para extrair dados de páginas web, OpenCV para o processamento de imagens e scikit-learn para converter os dados da imagens em informações importantes, que nesse caso é o conteúdo dos *CAPTCHAs*.

<center><img width=400 alt="computer_vision" src="https://miro.medium.com/max/1200/1*s9raSe9mLeSSuxE3API-ZA.gif"> <img width=300 alt="computer_vision" src="https://tulip.co/wp-content/uploads/barcode_ocr.gif"> <br>Fonte: https://towardsdatascience.com/how-to-do-everything-in-computer-vision-2b442c469928 e https://tulip.co/blog/tulip/give-your-apps-the-power-of-vision/
</center>

## Dependências

Nesse momento iremos instalar e importar as dependências necessárias para executar nosso notebook.

In [0]:
# Vamos instalar algumas bibliotecas e configurar algumas informações para o google colab. 

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# Importando as bibliotecas necessárias para o projeto
from selenium import webdriver # Para realizar o webscraping
from io import BytesIO # Para conversão das imagens na página para OpenCV
import matplotlib.pyplot as plt # Para mostrar os resultados dos processamentos
import cv2, numpy as np # Para manipular e realizar o processamento de imagens
import random # Para gerar número aleatórios
from joblib import load # Para importar o modelo de rede neural

SZ_W = 15
SZ_H = 20

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [0]:
# Montando o drive para poder acessar nossos arquivos.

from google.colab import drive
drive.mount('/content/drive')

# Definindo o caminho dos arquivos para o nosso projeto.
root_path = "drive/My Drive/campus-party-captchas"

---

<left><img width="100" alt="creating a repo" src="https://drive.google.com/uc?export=view&id=1E8tR7B9YYUXsU_rddJAyq0FrM0MSelxZ"></left>

## Codificação

<center> <h1>Selenium + Chrome webdriver</h1><img alt="webdriver-architecture" src="https://d1jnx9ba8s6j9r.cloudfront.net/blog/wp-content/uploads/2017/04/webdriver-working.png"> Fonte: https://www.edureka.co/blog/selenium-tutorial/
</center>


Selenium é uma biblioteca que nos permite navegar por páginas web e realizar ações **humanas** predefinidas, como preencher campos, ações de cliques e busca de termos na página. O que facilita as ações de *web crawling*, cujo propósito é criar scripts ou programas que automatizem tarefas.

### Inicializando o webdriver

In [0]:
# Instanciando nosso driver
driver = webdriver.Chrome('chromedriver',options=chrome_options)

### Realizando acesso a página

Nesse momento é realizado o acesso a página para obter a imagem do captcha.

In [0]:
# Definindo o caminho
url = 'http://captchas-generator.herokuapp.com/'

In [0]:
# Carregando a página solicitada
driver.get(url)

# Obtendo print da página
fullPage = driver.get_screenshot_as_png()

# Convertendo para Mat(OpenCV)
img_stream = BytesIO(fullPage)
img = cv2.imdecode(np.frombuffer(img_stream.read(), np.uint8), 1)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


In [0]:
# Mostrando a imagem carregada
plt.figure(figsize=(15,10))
plt.imshow(img)
plt.show

In [0]:
captcha = driver.find_element_by_id("captcha_image") # Encontrando o captcha na página pelo id
loc = captcha.location # Obtendo localização em pixels
size = captcha.size # Obtendo o tamanho da imagem
img = img[ int(loc['y']): int(loc['y'] + size['height']) , int(loc['x']): int(loc['x'] + size['width']) ] # Delimitando a imagem ao captcha
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convertendo para escala de cinza

In [0]:
# Mostrando o captcha obtido
plt.imshow(gray, cmap='gray')
plt.show

### Estágio de processamento de imagens

A partir desse ponto é iniciado a etapa de processamento.

#### Binarização

<center> <h1>Limiarização Adaptativa </h1><img alt="opencv_thresholding" src="https://docs.opencv.org/3.4.3/ada_threshold.jpg"><br> Fonte: https://docs.opencv.org/3.4.3/d7/d4d/tutorial_py_thresholding.html
</center>

O processo de limiarização consiste na transformação de uma imagem em escala de cinza para uma imagem binária. A limiarização mais simples é a definida por um **limite** predefinido, onde tudo que for menor que esse limite é transformado para um valor, e maior que ele para outro valor, ilustrado no topo esquerdo da Figura acima. 

Outros métodos mais complexos e mais eficientes também podem ser aplicados, como: <a href="https://en.wikipedia.org/wiki/Otsu%27s_method"> Otsu's method</a> e <a href="https://www.researchgate.net/publication/269984781_Adaptive_thresholding_A_comparative_study">limiarização adaptativa</a>. Este utiliza-se da vizinhaça para descobrir o limite, ao invés de um valor predefinido.

In [0]:
# Conversão da imagem em escala de cinza para preto e branco.
binary = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,75,10)
plt.imshow(binary, cmap='gray')
plt.show

#### Filtro de mediana

<center> <h1>Aplicação do filtro</h1><img alt="opencv_median" src="https://res.cloudinary.com/practicaldev/image/fetch/s--Wt8y6hP6--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_66%2Cw_880/https://cdn-images-1.medium.com/max/800/1%2AVyzJxDrLrC-OqkV5LHhjQQ.gif"><br> Fonte: https://dev.to/enzoftware/how-to-build-amazing-image-filters-with-python-median-filter---sobel-filter---5h7
</center>

O filtro de mediana é uns dos pré-processamentos mais utilizados na literatura quando o assunto é rúidos, principalmente os de sal e pipenta. 

<center><img width=300 alt="lena_normal" src="https://www.cosy.sbg.ac.at/~pmeerw/Watermarking/lena_gray.gif"> <img width=300 alt="lena_noise" src="https://raw.githubusercontent.com/timlentse/Add-Salt_Pepper_noise/master/add%20noise%20%20image.png">
</center>

Através de uma janela deslizante, o algoritmo busca o valor central (mediano) entre todos os valores dessa janela e o utiliza como valor do pixel para nova imagem. 

In [0]:
plt.figure(figsize=(20,5))
filtered = cv2.medianBlur(binary, 3) # Primeira aplicação
ax1 = plt.subplot(221)
ax1.imshow(filtered, cmap='gray')

filtered = cv2.medianBlur(filtered, 3) # Segunda aplicação
ax2 = plt.subplot(222)
ax2.imshow(filtered, cmap='gray')

filtered = cv2.medianBlur(filtered, 3) # Terceira aplicação
ax3 = plt.subplot(223)
ax3.imshow(filtered, cmap='gray')

filtered = cv2.medianBlur(filtered, 3) # Quarta aplicação
ax4 = plt.subplot(224)
ax4.imshow(filtered, cmap='gray')

ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')

ax1.text(0.5,-0.1, "Median blur 1", ha="center", transform=ax1.transAxes)
ax2.text(0.5,-0.1, "Median blur 2", ha="center", transform=ax2.transAxes)
ax3.text(0.5,-0.1, "Median blur 3", ha="center", transform=ax3.transAxes)
ax4.text(0.5,-0.1, "Final result", ha="center", transform=ax4.transAxes)

#### Buscando contornos

<center><h1>Encontrando contornos com OpenCV</h1><img alt="opencv_contour" src="https://www.pyimagesearch.com/wp-content/uploads/2016/01/center_of_contour_results.gif"><br> Fonte: https://www.pyimagesearch.com/2016/02/01/opencv-center-of-contour/
</center>

O processo de buscar contornos,consiste em analisar os pixels nas imagens e realizar o agrupamento desses pixels por meio da proximidade (distância e intensidade). Bastante utilizados na análise de silhuetas e na busca por objetos.

In [0]:
_, contours, _ = cv2.findContours(filtered, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Obtendo os contornos

img_cp = img.copy()

# Mostrando os contornos obtidos pelo algoritmo
for c in range(len(contours)):
    x,y,w,h = cv2.boundingRect(contours[c])
    cv2.rectangle(img_cp,(x,y),(x+w,y+h),
                 (random.randint(0, 255), 
                  random.randint(0, 255), 
                  random.randint(0, 255)),1)

plt.figure(figsize=(15,10))
plt.imshow(img_cp)
plt.show()

#### Ordenando pelo tamanho do contorno (número de pixels) e pela posição

In [0]:
# Definição da função que retorna o primeiro ponto a esquerda do contorno
def top_left(c):
    x,y,_,_ = cv2.boundingRect(c)
    return (x,y)

contours.sort(key=len,reverse=True) # Realizando a ordenação pelo tamanho (de maneira decrescente)
contours = contours[:min(6, len(contours))] # Obtendo os seis primeiros contornos
contours.sort(key=top_left) # Sorteando os 6 (ou menos) contornos pela posição

img_cp = img.copy()

# Mostrando os resultados finais
for c in range(len(contours)):
    x,y,w,h = cv2.boundingRect(contours[c])
    cv2.rectangle(img_cp,(x,y),(x+w,y+h),(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)),1)

plt.figure(figsize=(15,10))
plt.imshow(img_cp)
plt.show()

In [0]:
fig_chars = plt.figure()

# Separando e mostrando os candidatos a digitos
for c in range(len(contours)):
    x,y,w,h = cv2.boundingRect(contours[c])
    crop = np.zeros(filtered.shape, dtype=np.uint8)
    cv2.drawContours(crop, contours, c, 255, -1)
    crop = cv2.bitwise_and(crop, filtered)
    crop = crop[y:y+h, x:x+w]

    crop = cv2.resize(crop, (SZ_W, SZ_H), interpolation = cv2.INTER_CUBIC)
    fig_chars.add_subplot(1, 6, c+1)
    plt.imshow(crop, cmap = 'gray')

#### Corrigindo a orientação dos dígitos

<center><h1>Detectando inclinações</h1><img alt="opencv_skew" src="https://www.pyimagesearch.com/wp-content/uploads/2017/02/text_skew_pos41_results.png"><br> Fonte: https://www.pyimagesearch.com/2017/02/20/text-skew-correction-opencv-python/
</center>

A correção da inclinação textual é corrigida através de 2 momentos centrais, o qual é verificado a razão entre os dois eixos do plano.

In [0]:
# Definindo a função que corrige a orientação dos dígitos
def deskew(img):
    m = cv2.moments(img)
    
    if abs(m['mu02']) < 1e-2:
        return img.copy()    
    skew = m['mu11']/m['mu02']    
    M = np.float32([[1, skew, -0.5*SZ_W*skew], [0, 1, 0]])    
    img = cv2.warpAffine(img, M, (SZ_W, SZ_H), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_CUBIC)
    return img

In [0]:
fig_chars = plt.figure()
number_images = []

# Itera sobre os contornos e aplica a correção
for c in range(len(contours)):
    x,y,w,h = cv2.boundingRect(contours[c])
    crop = np.zeros(filtered.shape, dtype=np.uint8)
    cv2.drawContours(crop, contours, c, 255, -1)
    crop = cv2.bitwise_and(crop, filtered)
    crop = crop[y:y+h, x:x+w]

    crop = cv2.resize(crop, (SZ_W, SZ_H), interpolation = cv2.INTER_CUBIC) # Redimensionando o contorno para (15x20)
    skewed = deskew(crop) # Aplicação da correção
    fig_chars.add_subplot(1, 6, c+1)
    plt.imshow(skewed, cmap = 'gray')
    number_images.append( skewed.flatten().tolist() ) # Adiciona a lista de números os dígitos selecionados.

###  Classificação dos dígitos

Nessa etapa será realizado a classificação dos dígitos através de um modelo construido com aprendizado de máquina.

#### Rede neural

<center><h1>Arquitetura de uma RN</h1><img alt="opencv_skew" src="https://miro.medium.com/max/700/1*ui3IvoiVYBFtaU0auj63ew.gif"><br> Fonte: https://medium.com/@pallawi.ds/ai-starter-train-and-test-your-first-neural-network-classifier-in-keras-from-scratch-b6a5f3b3ebc4
</center>

Redes neurais são modelos computacionais compostos de
camadas de elementos de computação interconectados, os neurônios.
Elas são capazes de assimilar padrões em dados e, com isso,
realizar predições, diagnósticos, reconhecimentos, entre outros.


In [0]:
# Carregando o modelo de rede neural
mlp = load(root_path + '/models/mlp.save')

In [0]:
# Realizando a predição
mlp_result = mlp.predict(number_images)
mlp_result = ''.join( [str(x) for x in mlp_result] )
print(mlp_result)

## Finalizando o acesso à página


### Preenchendo os campos de texto

In [0]:
captcha_key = driver.find_element_by_id('captcha_input')
button = driver.find_element_by_id('captcha_btn')

### Submetendo os dados

In [0]:
captcha_key.send_keys(mlp_result)
button.click()

## Validando resultados

In [0]:
# Obtendo página de resultado
page = driver.page_source

In [0]:
try:
    success_message = driver.find_element_by_id('success') # Verifica a existência do id com a mensagem de sucesso
    print("Página carregada com sucesso!")
except:
    print("CAPTCHA inválida, tente novamente!") # Caso der errado

# Mostrando a página obtida
resultPage = driver.get_screenshot_as_png()
result_stream = BytesIO(resultPage)
result_img = cv2.imdecode(np.frombuffer(result_stream.read(), np.uint8), 1)
result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(15,10))
plt.imshow(result_img)
plt.show